In [1]:
import MDAnalysis as mda
import os
from MDAnalysis.analysis import lineardensity as lin
import matplotlib.pyplot as plt
from math import sqrt
import math
%matplotlib inline

In [2]:
current_directory = os.getcwd()
current_directory = str(current_directory)[:59]
data_files_directory_name = 'DataFiles'

graphene_water_psf = os.path.join(current_directory, data_files_directory_name, 'graphene_water.psf')
ns_graphene_water_dcd = os.path.join(current_directory, data_files_directory_name, '1ns.dcd')

#print(f'{graphene_water_psf} \n {ns_graphene_water_dcd}')

In [3]:
u = mda.Universe(graphene_water_psf,ns_graphene_water_dcd)
uni_SPCE = u.select_atoms('resname SPCE')
pos = uni_SPCE.positions

# (Max, Min) | X (14.4,-14.4) | Y (77, -77) | Z (79, 2)
# Axis of radius

c:\Users\drews\AppData\Local\Programs\Python\Python312\Lib\site-packages\MDAnalysis\coordinates\DCD.py:165: DeprecationWarning: DCDReader currently makes independent timesteps by copying self.ts while other readers update self.ts inplace. This behavior will be changed in 3.0 to be the same as other readers. Read more at https://github.com/MDAnalysis/mdanalysis/issues/3889 to learn if this change in behavior might affect you.
  warnings.warn("DCDReader currently makes independent timesteps"


In [4]:
# Find atoms where 2 <= z <= 10 for all x & y
# Find atoms where -8 <= y <= 8 for all x & z

unfiltered_position_dict = {'x_val':[],'y_val':[],'z_val':[]}

for coord in pos:
    unfiltered_position_dict['x_val'].append(coord[0])
    unfiltered_position_dict['y_val'].append(coord[1])
    unfiltered_position_dict['z_val'].append(coord[2])

x_min,y_min,z_min = min(unfiltered_position_dict['x_val']),min(unfiltered_position_dict['y_val']),min(unfiltered_position_dict['z_val'])

filtered_position_dict = {'x_val':[],'y_val':[],'z_val':[]}

for coord in pos:
    filtered_position_dict['x_val'].append(abs(coord[0]-x_min))
    filtered_position_dict['y_val'].append(abs(coord[1]-y_min))
    filtered_position_dict['z_val'].append(abs(coord[2]-z_min))

print(len(filtered_position_dict['x_val']))

24000


In [12]:
approximating_percentage = .20
approximate_y = max(filtered_position_dict['y_val']) * approximating_percentage
approximate_z = max(filtered_position_dict['z_val']) * approximating_percentage
approximate_volume = max(filtered_position_dict['x_val']) * approximate_y * approximate_z

In [13]:
heights = []
distances = []

for iteration in range(int(1/approximating_percentage)):
    heights.append(iteration * approximate_z)
    distances.append(iteration * approximate_y)

print(len(heights))

5


In [19]:
atomic_coords = {}

for i in range(len(distances)):
    atomic_coords[i] = {}
    atomic_coords[i]['y_val'] = []
    atomic_coords[i]['z_val'] = []


print(atomic_coords)

for j in range(len(filtered_position_dict['y_val']) - 1):
    for i in range(len(heights)-1):

        if distances[i] <= filtered_position_dict['y_val'][j] and filtered_position_dict['y_val'][j] <= distances[i+1] and filtered_position_dict['z_val'][j] <= heights[1]:
                print(j, atomic_coords[i]['y_val'][j])
                atomic_coords[i]['y_val'].append()
                atomic_coords[i]['z_val'].append()
print(atomic_coords)


{0: {'y_val': [], 'z_val': []}, 1: {'y_val': [], 'z_val': []}, 2: {'y_val': [], 'z_val': []}, 3: {'y_val': [], 'z_val': []}, 4: {'y_val': [], 'z_val': []}}


IndexError: list index out of range

In [8]:
iz_axy, iy_azx = [], []

#Initial bounded z, addition of bounded x,y
initial_z_bound1, initial_z_bound2 = 5, 10
chosen_y_bound1, chosen_y_bound2 = -10, 10

for coord in pos:
    if (initial_z_bound1 <= coord[2] and coord[2] <= initial_z_bound2) and (chosen_y_bound1 <= coord[1] and coord[1] <= chosen_y_bound2):
        iz_axy.append(coord)
volume_initial_bounded_z = (abs(initial_z_bound2 - initial_z_bound1)) * (abs(chosen_y_bound2 - chosen_y_bound1)) * (abs(x_max - x_min))

#Initial bounded y, addition of bounded x,z
initial_y_bound1, initial_y_bound2 = -8, 8
chosen_z_bound1, chosen_z_bound2 = 5, 15

for coord in pos:
    if (chosen_z_bound1 <= coord[2] and coord[2] <= chosen_z_bound2) and (initial_y_bound1 <= coord[1] and coord[1] <= initial_y_bound2):
        iy_azx.append(coord)
volume_initial_bounded_y = (abs(initial_y_bound2 - initial_y_bound1)) * (abs(chosen_z_bound2 - chosen_z_bound1)) * (abs(x_max - x_min))


NameError: name 'x_max' is not defined

In [ ]:
mass_h2o = 18.02

iz_density = len(iz_axy) * 18.02 / volume_initial_bounded_z
iy_density = len(iy_azx) * 18.02 / volume_initial_bounded_y

print(f'iz: {iz_density} | iy: {iy_density}')

iz: 1.7281026731616396 | iy: 1.7895953777723186


In [ ]:
total_x_length = (max(x_coords)-min(x_coords))
total_y_length = (max(y_coords)-min(y_coords))
total_z_length = (max(z_coords)-min(z_coords))
total_volume = total_x_length * total_y_length * total_z_length
print(f'Totals | X = {round(float(total_x_length),2)} | Y = {round(float(total_y_length),2)} | Z = {round(float(total_z_length),2)} | V = {round(float(total_volume),2)}')

Totals | X = 28.57 | Y = 152.9 | Z = 76.02 | V = 332106.62


7.644920349121094 3.8010955810546876


In [ ]:
output_coords = {}

for heights in range(int(1/approximating_percentage)):
    output_coords[heights] = {}
    for iteration in range(int(1/approximating_percentage)):
        print(iteration)
        output_coords[heights][iteration] = []

        for coord in pos:
            if (iteration) * approximate_y <= coord[1] and coord[1] <= (iteration + 1) * approximate_y and (heights) * approximate_z <= coord[2] and coord[2] <= (heights + 1 ) * approximate_z:
                output_coords[heights][iteration].append(coord)


y_coords,z_coords = [],[]

container_len = []
for key in output_coords:
    container_len.append(len(output_coords[key]))

#Height | Iteration
#0 - 1  | 11
#1 - 2  | 11
#2 - 3  | 11
#3 - 4  | 11


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [ ]:
#print(range(int(1/approximating_percentage)))
#print(container_len)
#print(len(uni_SPCE))
#print(output_coords)

for height in output_coords[heights]:
    print(output_coords[heights])

{0: [array([ 0.5586996,  4.157745 , 74.64071  ], dtype=float32), array([-0.4193899,  4.2011247, 74.84432  ], dtype=float32), array([ 0.7761002,  4.8050466, 73.91013  ], dtype=float32), array([ 0.87823373,  6.155552  , 72.80848   ], dtype=float32), array([ 0.49045685,  6.953876  , 73.26925   ], dtype=float32), array([ 8.17203   ,  0.48296177, 75.04768   ], dtype=float32), array([ 7.9772134,  1.2186087, 74.39894  ], dtype=float32), array([ 8.460513  ,  0.87616336, 75.92071   ], dtype=float32), array([-10.433968 ,   0.5555115,  73.59279  ], dtype=float32), array([-11.143121  ,   0.29775366,  74.24904   ], dtype=float32), array([-10.665195  ,   0.18963572,  72.69131   ], dtype=float32), array([-4.711833 ,  1.5372796, 72.62473  ], dtype=float32), array([-4.836054  ,  0.55924916, 72.45732   ], dtype=float32), array([-4.7356105,  4.9145236, 72.30276  ], dtype=float32), array([-4.71682 ,  4.517712, 73.22047 ], dtype=float32), array([-8.676153  ,  0.28545725, 75.6961    ], dtype=float32), array

In [ ]:
output_coords = {}

for iteration in range(int(1 / approximating_percentage)):
    output_coords[iteration] = []
    for coord in pos:
        if (iteration - 1 ) * approximate_y <= coord[1] and coord[1] <= iteration * approximate_y and coord[2] <= approximate_z:
            output_coords[iteration].append(coord)
print(output_coords[0])

[array([-8.143595 , -3.160693 ,  3.6604617], dtype=float32), array([-5.638383 , -3.9457512,  3.7287333], dtype=float32), array([-3.7468078, -1.2099922,  3.6956105], dtype=float32), array([-2.7651603, -1.0213695,  3.6675084], dtype=float32), array([-3.9017262, -2.1897414,  3.568758 ], dtype=float32), array([-3.7217135, -5.001293 ,  3.5742838], dtype=float32), array([-9.377103 , -0.9717005,  3.7773051], dtype=float32), array([-1.0355884 , -0.76170856,  3.7282598 ], dtype=float32), array([-0.13275072, -1.1506377 ,  3.5449066 ], dtype=float32), array([ 8.917307 , -0.8724833,  3.5724242], dtype=float32), array([ 1.5346296, -1.4513543,  3.552313 ], dtype=float32), array([ 2.1070118 , -0.63156134,  3.5344775 ], dtype=float32), array([ 5.698984 , -2.2415166,  3.5466583], dtype=float32), array([ 6.593073 , -1.8249837,  3.7112954], dtype=float32), array([ 5.455986 , -2.1388736,  2.5820773], dtype=float32), array([-1.3446631, -7.556462 ,  2.5322852], dtype=float32), array([10.623947 , -1.0414734,